In [2]:
#importing CSV File and Reading it
import pandas as pd
import csv

df= pd.read_csv("002_AppendixD.tsv", sep='\t', encoding='latin-1')
df

,Country,GHI1992,GHI2000,GHI2008,GHI2017
0,Afghanistan,50.2,52.7,37.9,33.3
1,Albania,20.8,21.6,16.5,11.1
2,Algeria,17.5,15.6,11.3,9.5
3,Angola,65.8,57.5,39.7,32.5
4,Argentina,7,6.6,5.8,5.4
5,Armenia,-,18.4,11.4,7.7
6,Azerbaijan,-,27.5,15.3,9.6
7,Bahrain,-,-,-,-
8,Bangladesh,53.6,37.6,32.2,26.5
9,Belarus,-,<5,<5,<5


In [3]:
df2= pd.read_csv("001_AppendixC.tsv", sep='\t', encoding='latin-1')
df2

,Country,UN9193,UN9901,UN0709,UN1416,stu9094,stu9802,stu0610,stu1216,wast9094,wast9802,wast0610,wast1216,UM1992,UM2000,UM2008,UM2015
0,Afghanistan,37.4,46.1,24.6,23,51.3,54.7,51.3,40.9,11.5,13.4,8.9,9.5,16.8,13.7,11,9.1
1,Albania,7.7,7.2,9.6999999999999993,4.9000000000000004,37.200000000000003,39.200000000000003,23.1,17.899999999999999,9.4,12.2,9.4,6.2,3.7,2.6,1.8,1.4
2,Algeria,9.1,10.7,7.5,4.5999999999999996,22.9,23.6,13.2,11.7,7.1,3.1,4.0999999999999996,4.0999999999999996,4.5,4,2.9,2.6
3,Angola,65.400000000000006,50,23.7,14,52.9,47.1,29.2,37.6,7.9,8.6999999999999993,8.1999999999999993,4.9000000000000004,22.6,21.7,19.2,15.7
4,Argentina,4.7,3.5,4.0999999999999996,3.6,7.1,9.9,8.1999999999999993,7.5,1.6,1.6,1.2,1.6,2.6,2,1.5,1.3
5,Armenia,-,23.8,5.2,4.4000000000000004,23.5,17.7,20.8,9.4,5,2.5,4.2,4.2,4.5,3,2,1.4
6,Azerbaijan,-,23.2,2.6,1.2,30,24.1,26.8,18,6.6,9,6.8,3.1,9.5,7.4,4.3,3.2
7,Bahrain,-,-,-,-,15.3,13.7,9.1999999999999993,9.1,7,6.7,3.5,3.1,2.2000000000000002,1.3,0.9,0.6
8,Bangladesh,36.1,20.8,16.399999999999999,15.1,71.5,54,43.2,36.1,16.100000000000001,13.8,17.5,14.3,13.2,8.8000000000000007,5.6,3.8
9,Belarus,-,2.2999999999999998,1.6,1.1000000000000001,7.5,6,4.2,2.8,2.6,2,1.9,1.8,1.7,1.4,0.7,0.5
